In [1]:
import argparse
import datetime
import glob
import hashlib
import json
import multiprocessing
import pickle
import os
import shutil
import subprocess
import uuid
import random

import numpy as np
import pandas as pd
import lightgbm as lgb
from collections import defaultdict
from lightgbm.sklearn import LGBMRanker
from tqdm import tqdm

from pyserini.analysis import Analyzer, get_lucene_analyzer
from pyserini.ltr import *
from pyserini.search import get_topics_with_reader

def train_data_loader(task='triple', neg_sample=10, random_seed=12345):
    if os.path.exists(f'train_{task}_sampled_with_{neg_sample}_{random_seed}.pickle'):
        sampled_train = pd.read_pickle(f'train_{task}_sampled_with_{neg_sample}_{random_seed}.pickle')
        print(sampled_train.shape)
        print(sampled_train.index.get_level_values('qid').drop_duplicates().shape)
        print(sampled_train.groupby('qid').count().mean())
        print(sampled_train.head(10))
        print(sampled_train.info())
        return sampled_train
    else:
        if task == 'triple':
            train = pd.read_csv('../collections/msmarco-passage/qidpidtriples.train.full.2.tsv', sep="\t",
                                names=['qid', 'pos_pid', 'neg_pid'], dtype=np.int32)
            pos_half = train[['qid', 'pos_pid']].rename(columns={"pos_pid": "pid"}).drop_duplicates()
            pos_half['rel'] = np.int32(1)
            neg_half = train[['qid', 'neg_pid']].rename(columns={"neg_pid": "pid"}).drop_duplicates()
            neg_half['rel'] = np.int32(0)
            del train
            sampled_neg_half = []
            for qid, group in tqdm(neg_half.groupby('qid')):
                sampled_neg_half.append(group.sample(n=min(neg_sample, len(group)), random_state=random_seed))
            sampled_train = pd.concat([pos_half] + sampled_neg_half, axis=0, ignore_index=True)
            sampled_train = sampled_train.sort_values(['qid','pid']).set_index(['qid','pid'])
            print(sampled_train.shape)
            print(sampled_train.index.get_level_values('qid').drop_duplicates().shape)
            print(sampled_train.groupby('qid').count().mean())
            print(sampled_train.head(10))
            print(sampled_train.info())

            sampled_train.to_pickle(f'train_{task}_sampled_with_{neg_sample}_{random_seed}.pickle')
        elif task == 'rank':
            qrel = defaultdict(list)
            with open("../collections/msmarco-passage/qrels.train.tsv") as f:
                for line in f:
                    topicid, _, docid, rel = line.strip().split('\t')
                    assert rel == "1", line.split(' ')
                    qrel[topicid].append(docid)
            
            qid2pos = defaultdict(list)
            qid2neg = defaultdict(list)
            with open("../runs/msmarco-passage-expanded/run.msmarco-passage-expanded.train.small.txt") as f:
                for line in tqdm(f):
                    topicid, _, docid, rank, _, _ = line.split()
                    assert topicid in qrel
                    if docid in qrel[topicid]:
                        qid2pos[topicid].append(docid)
                    else:
                        qid2neg[topicid].append(docid)
            sampled_train = []
            for topicid, pos_list in tqdm(qid2pos.items()):
                neg_list = random.sample(qid2neg[topicid], min(len(qid2neg[topicid]), neg_sample))
                for positive_docid in pos_list:
                    sampled_train.append((int(topicid), int(positive_docid), 1))
                for negative_docid in neg_list:
                    sampled_train.append((int(topicid), int(negative_docid), 0))
            sampled_train = pd.DataFrame(sampled_train,columns=['qid','pid','rel'],dtype=np.int32)
            sampled_train = sampled_train.sort_values(['qid','pid']).set_index(['qid','pid'])
            print(sampled_train.shape)
            print(sampled_train.index.get_level_values('qid').drop_duplicates().shape)
            print(sampled_train.groupby('qid').count().mean())
            print(sampled_train.head(10))
            print(sampled_train.info())

            sampled_train.to_pickle(f'train_{task}_sampled_with_{neg_sample}_{random_seed}.pickle')
        else:
            raise Exception('unknown parameters')
        return sampled_train
sampled_train = train_data_loader(task='rank', neg_sample=10)

502768028it [08:26, 992005.00it/s] 
100%|██████████| 479348/479348 [00:13<00:00, 34369.57it/s]


(5298347, 1)
(479348,)
rel    11.053237
dtype: float64
             rel
qid pid         
3   1142680    1
    2987752    0
    3108851    0
    3317747    0
    4268102    0
    6259556    0
    6291853    0
    6305078    0
    7312206    0
    7441942    0
<class 'pandas.core.frame.DataFrame'>
MultiIndex: 5298347 entries, (3, 1142680) to (1185869, 7204001)
Data columns (total 1 columns):
 #   Column  Dtype
---  ------  -----
 0   rel     int32
dtypes: int32(1)
memory usage: 272.2 MB
None


In [2]:
def dev_data_loader(task='pygaggle'):
    if os.path.exists(f'dev_{task}.pickle'):
        dev = pd.read_pickle(f'dev_{task}.pickle')
        print(dev.shape)
        print(dev.index.get_level_values('qid').drop_duplicates().shape)
        print(dev.groupby('qid').count().mean())
        print(dev.head(10))
        print(dev.info())
        dev_qrel = pd.read_pickle(f'dev_qrel.pickle')
        return dev, dev_qrel
    else:
        if task == 'rerank':
            dev = pd.read_csv('../collections/msmarco-passage/top1000.dev', sep="\t",
                              names=['qid', 'pid', 'query', 'doc'], usecols=['qid', 'pid'], dtype=np.int32)
        elif task == 'anserini':
            dev = pd.read_csv('../runs/msmarco-passage-expanded/run.msmarco-passage-expanded.dev.small.txt',sep="\t",
                            names=['qid','pid','rank'], dtype=np.int32)
        else:
            raise Exception('unknown parameters')
        dev_qrel = pd.read_csv('../collections/msmarco-passage/qrels.dev.small.tsv', sep="\t",
                               names=["qid", "q0", "pid", "rel"], usecols=['qid', 'pid', 'rel'], dtype=np.int32)
        dev = dev.merge(dev_qrel, left_on=['qid', 'pid'], right_on=['qid', 'pid'], how='left')
        dev['rel'] = dev['rel'].fillna(0).astype(np.int32)
        dev = dev.sort_values(['qid','pid']).set_index(['qid','pid'])
        
        print(dev.shape)
        print(dev.index.get_level_values('qid').drop_duplicates().shape)
        print(dev.groupby('qid').count().mean())
        print(dev.head(10))
        print(dev.info())

        dev.to_pickle(f'dev_{task}.pickle')
        dev_qrel.to_pickle(f'dev_qrel.pickle')
        return dev, dev_qrel
dev, dev_qrel = dev_data_loader(task='anserini')

(6974999, 2)
(6980,)
rank    999.283524
rel     999.283524
dtype: float64
           rank  rel
qid pid             
2   35571   745    0
    55860   657    0
    62497   354    0
    63138   979    0
    72210   961    0
    98587   316    0
    98589   256    0
    98590   425    0
    98592   637    0
    98595   340    0
<class 'pandas.core.frame.DataFrame'>
MultiIndex: 6974999 entries, (2, 35571) to (1102400, 8841784)
Data columns (total 2 columns):
 #   Column  Dtype
---  ------  -----
 0   rank    int32
 1   rel     int32
dtypes: int32(2)
memory usage: 286.8 MB
None


In [3]:
def query_loader(choice='default'):
    if os.path.exists(f'query_{choice}_tokenized.pickle'):
        return pickle.load(open(f'query_{choice}_tokenized.pickle','rb'))
    else:
        if choice == 'default':
            analyzer = Analyzer(get_lucene_analyzer())
            nonStopAnalyzer = Analyzer(get_lucene_analyzer(stopwords=False))
            queries = get_topics_with_reader('io.anserini.search.topicreader.TsvIntTopicReader', \
                                             '../collections/msmarco-passage/queries.train.tsv')
            queries.update(get_topics_with_reader('io.anserini.search.topicreader.TsvIntTopicReader', \
                                                  '../collections/msmarco-passage/queries.dev.tsv'))
            for qid,value in queries.items():
                assert 'tokenized' not in value
                value['tokenized'] = analyzer.analyze(value['title'])
                assert 'nonSW' not in value
                value['nonSW'] = nonStopAnalyzer.analyze(value['title'])
        else:
            raise Exception('unknown parameters')

        pickle.dump(queries,open(f'query_{choice}_tokenized.pickle','wb'))

        return queries
queries = query_loader()

In [4]:
fe = FeatureExtractor('../indexes/msmarco-passage-expanded/lucene-index-msmarco-passage-expanded/',
                      max(multiprocessing.cpu_count()//2,1))
fe.add(BM25(k1=0.82,b=0.68))
fe.add(BM25(k1=0.9,b=0.4))
fe.add(BM25(k1=1.2,b=0.75))
fe.add(BM25(k1=2.0,b=0.75))

fe.add(LMDir(mu=1000))
fe.add(LMDir(mu=1500))
fe.add(LMDir(mu=2500))

fe.add(LMJM(0.1))
fe.add(LMJM(0.4))
fe.add(LMJM(0.7))

fe.add(NTFIDF())
fe.add(ProbalitySum())

fe.add(DFR_GL2())
fe.add(DFR_In_expB2())
fe.add(DPH())

fe.add(Proximity())
fe.add(TPscore())
fe.add(tpDist())

fe.add(DocSize())

fe.add(QueryLength())
fe.add(QueryLengthNonStopWords())
fe.add(QueryCoverageRatio())
fe.add(UniqueTermCount())
fe.add(MatchingTermCount())
fe.add(SCS())

fe.add(tfStat(AvgPooler()))
fe.add(tfStat(MedianPooler()))
fe.add(tfStat(SumPooler()))
fe.add(tfStat(MinPooler()))
fe.add(tfStat(MaxPooler()))
fe.add(tfStat(VarPooler()))
fe.add(tfStat(MaxMinRatioPooler()))
fe.add(tfStat(ConfidencePooler()))

fe.add(tfIdfStat(AvgPooler()))
fe.add(tfIdfStat(MedianPooler()))
fe.add(tfIdfStat(SumPooler()))
fe.add(tfIdfStat(MinPooler()))
fe.add(tfIdfStat(MaxPooler()))
fe.add(tfIdfStat(VarPooler()))
fe.add(tfIdfStat(MaxMinRatioPooler()))
fe.add(tfIdfStat(ConfidencePooler()))

fe.add(scqStat(AvgPooler()))
fe.add(scqStat(MedianPooler()))
fe.add(scqStat(SumPooler()))
fe.add(scqStat(MinPooler()))
fe.add(scqStat(MaxPooler()))
fe.add(scqStat(VarPooler()))
fe.add(scqStat(MaxMinRatioPooler()))
fe.add(scqStat(ConfidencePooler()))

fe.add(normalizedTfStat(AvgPooler()))
fe.add(normalizedTfStat(MedianPooler()))
fe.add(normalizedTfStat(SumPooler()))
fe.add(normalizedTfStat(MinPooler()))
fe.add(normalizedTfStat(MaxPooler()))
fe.add(normalizedTfStat(VarPooler()))
fe.add(normalizedTfStat(MaxMinRatioPooler()))
fe.add(normalizedTfStat(ConfidencePooler()))

fe.add(idfStat(AvgPooler()))
fe.add(idfStat(MedianPooler()))
fe.add(idfStat(SumPooler()))
fe.add(idfStat(MinPooler()))
fe.add(idfStat(MaxPooler()))
fe.add(idfStat(VarPooler()))
fe.add(idfStat(MaxMinRatioPooler()))
fe.add(idfStat(ConfidencePooler()))

fe.add(ictfStat(AvgPooler()))
fe.add(ictfStat(MedianPooler()))
fe.add(ictfStat(SumPooler()))
fe.add(ictfStat(MinPooler()))
fe.add(ictfStat(MaxPooler()))
fe.add(ictfStat(VarPooler()))
fe.add(ictfStat(MaxMinRatioPooler()))
fe.add(ictfStat(ConfidencePooler()))

fe.add(UnorderedSequentialPairs(3))
fe.add(UnorderedSequentialPairs(8))
fe.add(UnorderedSequentialPairs(15))
fe.add(OrderedSequentialPairs(3))
fe.add(OrderedSequentialPairs(8))
fe.add(OrderedSequentialPairs(15))
fe.add(UnorderedQueryPairs(3))
fe.add(UnorderedQueryPairs(8))
fe.add(UnorderedQueryPairs(15))
fe.add(OrderedQueryPairs(3))
fe.add(OrderedQueryPairs(8))
fe.add(OrderedQueryPairs(15))

fe.add(BM25(k1=0.82,b=0.68, field='expansion'))
fe.add(BM25(k1=0.9,b=0.4, field='expansion'))
fe.add(BM25(k1=1.2,b=0.75, field='expansion'))
fe.add(BM25(k1=2.0,b=0.75, field='expansion'))

fe.add(LMDir(mu=1000, field='expansion'))
fe.add(LMDir(mu=1500, field='expansion'))
fe.add(LMDir(mu=2500, field='expansion'))

fe.add(LMJM(0.1, field='expansion'))
fe.add(LMJM(0.4, field='expansion'))
fe.add(LMJM(0.7, field='expansion'))

fe.add(NTFIDF(field='expansion'))
fe.add(ProbalitySum(field='expansion'))

fe.add(DFR_GL2(field='expansion'))
fe.add(DFR_In_expB2(field='expansion'))
fe.add(DPH(field='expansion'))

fe.add(Proximity(field='expansion'))
fe.add(TPscore(field='expansion'))
fe.add(tpDist(field='expansion'))

fe.add(DocSize(field='expansion'))

fe.add(QueryCoverageRatio(field='expansion'))
fe.add(MatchingTermCount(field='expansion'))
fe.add(SCS(field='expansion'))

fe.add(tfStat(AvgPooler(),field='expansion'))
fe.add(tfStat(MedianPooler(),field='expansion'))
fe.add(tfStat(SumPooler(),field='expansion'))
fe.add(tfStat(MinPooler(),field='expansion'))
fe.add(tfStat(MaxPooler(),field='expansion'))
fe.add(tfStat(VarPooler(),field='expansion'))
fe.add(tfStat(MaxMinRatioPooler(),field='expansion'))
fe.add(tfStat(ConfidencePooler(),field='expansion'))

fe.add(tfIdfStat(AvgPooler(),field='expansion'))
fe.add(tfIdfStat(MedianPooler(),field='expansion'))
fe.add(tfIdfStat(SumPooler(),field='expansion'))
fe.add(tfIdfStat(MinPooler(),field='expansion'))
fe.add(tfIdfStat(MaxPooler(),field='expansion'))
fe.add(tfIdfStat(VarPooler(),field='expansion'))
fe.add(tfIdfStat(MaxMinRatioPooler(),field='expansion'))
fe.add(tfIdfStat(ConfidencePooler(),field='expansion'))

fe.add(scqStat(AvgPooler(),field='expansion'))
fe.add(scqStat(MedianPooler(),field='expansion'))
fe.add(scqStat(SumPooler(),field='expansion'))
fe.add(scqStat(MinPooler(),field='expansion'))
fe.add(scqStat(MaxPooler(),field='expansion'))
fe.add(scqStat(VarPooler(),field='expansion'))
fe.add(scqStat(MaxMinRatioPooler(),field='expansion'))
fe.add(scqStat(ConfidencePooler(),field='expansion'))

fe.add(normalizedTfStat(AvgPooler(),field='expansion'))
fe.add(normalizedTfStat(MedianPooler(),field='expansion'))
fe.add(normalizedTfStat(SumPooler(),field='expansion'))
fe.add(normalizedTfStat(MinPooler(),field='expansion'))
fe.add(normalizedTfStat(MaxPooler(),field='expansion'))
fe.add(normalizedTfStat(VarPooler(),field='expansion'))
fe.add(normalizedTfStat(MaxMinRatioPooler(),field='expansion'))
fe.add(normalizedTfStat(ConfidencePooler(),field='expansion'))

fe.add(idfStat(AvgPooler(),field='expansion'))
fe.add(idfStat(MedianPooler(),field='expansion'))
fe.add(idfStat(SumPooler(),field='expansion'))
fe.add(idfStat(MinPooler(),field='expansion'))
fe.add(idfStat(MaxPooler(),field='expansion'))
fe.add(idfStat(VarPooler(),field='expansion'))
fe.add(idfStat(MaxMinRatioPooler(),field='expansion'))
fe.add(idfStat(ConfidencePooler(),field='expansion'))

fe.add(ictfStat(AvgPooler(),field='expansion'))
fe.add(ictfStat(MedianPooler(),field='expansion'))
fe.add(ictfStat(SumPooler(),field='expansion'))
fe.add(ictfStat(MinPooler(),field='expansion'))
fe.add(ictfStat(MaxPooler(),field='expansion'))
fe.add(ictfStat(VarPooler(),field='expansion'))
fe.add(ictfStat(MaxMinRatioPooler(),field='expansion'))
fe.add(ictfStat(ConfidencePooler(),field='expansion'))

fe.add(UnorderedSequentialPairs(3,field='expansion'))
fe.add(UnorderedSequentialPairs(8,field='expansion'))
fe.add(UnorderedSequentialPairs(15,field='expansion'))
fe.add(OrderedSequentialPairs(3,field='expansion'))
fe.add(OrderedSequentialPairs(8,field='expansion'))
fe.add(OrderedSequentialPairs(15,field='expansion'))
fe.add(UnorderedQueryPairs(3,field='expansion'))
fe.add(UnorderedQueryPairs(8,field='expansion'))
fe.add(UnorderedQueryPairs(15,field='expansion'))
fe.add(OrderedQueryPairs(3,field='expansion'))
fe.add(OrderedQueryPairs(8,field='expansion'))
fe.add(OrderedQueryPairs(15,field='expansion'))

In [5]:
def extract(df, queries, fe):
    df_pieces = []
    fetch_later = []
    qidpid2rel = defaultdict(dict)
    need_rows = 0
    for qid,group in tqdm(df.groupby('qid')):
        for t in group.reset_index().itertuples():
            assert t.pid not in qidpid2rel[t.qid]
            qidpid2rel[t.qid][t.pid] = t.rel
            need_rows += 1
        #test.py has bug here, it does not convert pid to str, not sure why it does not cause problem in java
        fe.lazy_extract(str(qid),
                        queries[qid]['nonSW'], 
                        queries[qid]['tokenized'],
                        [str(pid) for pid in qidpid2rel[t.qid].keys()])
        fetch_later.append(str(qid))
        if len(fetch_later) == 10000:
            info = np.zeros(shape=(need_rows,3), dtype=np.int32)
            feature = np.zeros(shape=(need_rows,len(fe.feature_names())), dtype=np.float32)
            idx = 0
            for qid in fetch_later:
                for doc in fe.get_result(qid):
                    info[idx,0] = int(qid)
                    info[idx,1] = int(doc['pid'])
                    info[idx,2] = qidpid2rel[int(qid)][int(doc['pid'])]
                    feature[idx,:] = doc['features']
                    idx += 1
            info = pd.DataFrame(info, columns=['qid','pid','rel'])
            feature = pd.DataFrame(feature, columns=fe.feature_names())
            df_pieces.append(pd.concat([info,feature], axis=1))
            fetch_later = []
            need_rows = 0
    #deal with rest
    if len(fetch_later) > 0:
        info = np.zeros(shape=(need_rows,3), dtype=np.int32)
        feature = np.zeros(shape=(need_rows,len(fe.feature_names())), dtype=np.float32)
        idx = 0
        for qid in fetch_later:
            for doc in fe.get_result(qid):
                info[idx,0] = int(qid)
                info[idx,1] = int(doc['pid'])
                info[idx,2] = qidpid2rel[int(qid)][int(doc['pid'])]
                feature[idx,:] = doc['features']
                idx += 1
        info = pd.DataFrame(info, columns=['qid','pid','rel'])
        feature = pd.DataFrame(feature, columns=fe.feature_names())
        df_pieces.append(pd.concat([info,feature], axis=1))
    data = pd.concat(df_pieces, axis=0, ignore_index=True)
    data = data.sort_values(by='qid', kind='mergesort')
    group = data.groupby('qid').agg(count=('pid', 'count'))['count']
    return data,group

In [6]:
def hash_df(df):
    h = pd.util.hash_pandas_object(df)
    return hex(h.sum().astype(np.uint64))


def hash_anserini_jar():
    find = glob.glob(os.environ['ANSERINI_CLASSPATH'] + "/*fatjar.jar")
    assert len(find) == 1
    md5Hash = hashlib.md5(open(find[0], 'rb').read())
    return md5Hash.hexdigest()


def hash_fe(fe):
    return hashlib.md5(','.join(sorted(fe.feature_names())).encode()).hexdigest()


def data_loader(task, df, queries, fe):
    df_hash = hash_df(df)
    jar_hash = hash_anserini_jar()
    fe_hash = hash_fe(fe)
    print(f'check {task}_{df_hash}_{jar_hash}_{fe_hash}.pickle')
    if os.path.exists(f'{task}_{df_hash}_{jar_hash}_{fe_hash}.pickle'):
        res = pickle.load(open(f'{task}_{df_hash}_{jar_hash}_{fe_hash}.pickle','rb'))
        print(res['data'].shape)
        print(res['data'].qid.drop_duplicates().shape)
        print(res['group'].mean())
        print(res['data'].head(10))
        print(res['data'].info())
        return res
    else:
        if task == 'train' or task == 'dev': 
            data,group = extract(df, queries, fe)
            obj = {'data':data,'group':group,'df_hash':df_hash,'jar_hash':jar_hash,'fe_hash':fe_hash}
            print(data.shape)
            print(data.qid.drop_duplicates().shape)
            print(group.mean())
            print(data.head(10))
            print(data.info())
            pickle.dump(obj,open(f'{task}_{df_hash}_{jar_hash}_{fe_hash}.pickle','wb'))
            return obj
        else:
            raise Exception('unknown parameters')

In [7]:
import json
def export(df, fn):
    with open(fn,'w') as f:
        for qid, group in tqdm(df.groupby('qid')):
            line = {}
            line['qid'] = qid
            line['queryTokens'] = queries[qid]['tokenized']
            line['queryText'] = queries[qid]['nonSW']
            line['docIds'] = [str(did) for did in group.reset_index().pid.drop_duplicates().tolist()]
            f.write(json.dumps(line)+'\n')

In [8]:
train_extracted = data_loader('train', sampled_train, queries, fe)
# export(sampled_train, 'sampled_train_export.json')
dev_extracted = data_loader('dev', dev, queries, fe)
# export(dev, 'sampled_dev_export.json')
del sampled_train, dev
feature_name = fe.feature_names()
del queries, fe

check train_0xcc3dbba0c62c8ae_7e428e7b5ff963bf9aa2c464acf087bd_899f9b1fc7fe20a8748222d471b2b5f5.pickle


100%|██████████| 479348/479348 [23:09<00:00, 344.97it/s]  


(5298347, 170)
(479348,)
11.0532368967848
   qid      pid  rel  contents_BM25_k1_0.82_b_0.68  \
0    3  1142680    1                     25.378399   
1    3  2987752    0                     16.366947   
2    3  3108851    0                     21.005810   
3    3  3317747    0                     17.289114   
4    3  4268102    0                     23.480787   
5    3  6259556    0                     16.782444   
6    3  6291853    0                     12.538409   
7    3  6305078    0                     16.144419   
8    3  7312206    0                     11.398890   
9    3  7441942    0                     13.343893   

   contents_BM25_k1_0.90_b_0.40  contents_BM25_k1_1.20_b_0.75  \
0                     26.467836                     27.822981   
1                     14.889225                     17.930616   
2                     20.519165                     22.139957   
3                     16.312189                     19.101547   
4                     24.633562       

In [9]:
def eval_mrr(dev_data):
    score_tie_counter = 0
    score_tie_query = set()

    MRR = []
    for qid, group in tqdm(dev_data.groupby('qid')):
        group = group.reset_index()
        rank = 0
        prev_score = None
        assert len(group['pid'].tolist()) == len(set(group['pid'].tolist()))
        # stable sort is also used in LightGBM

        for t in group.sort_values('score', ascending=False, kind='mergesort').itertuples():
            if prev_score is not None and abs(t.score - prev_score) < 1e-8:
                score_tie_counter += 1
                score_tie_query.add(qid)
            prev_score = t.score
            prev_pid = t.pid
            rank += 1
            if t.rel>0:
                MRR.append(1.0/rank)
                break
            elif rank == 10 or rank == len(group):
                MRR.append(0.)
                break

    score_tie = f'score_tie occurs {score_tie_counter} times in {len(score_tie_query)} queries'
    print(score_tie)
    print(np.mean(MRR))


In [10]:
def eval_recall(dev_qrel, dev_data):
    dev_rel_num = dev_qrel[dev_qrel['rel']>0].groupby('qid').count()['rel']

    score_tie_counter = 0
    score_tie_query = set()
    
    recall_point = [10,20,50,100,200,500,1000]
    recall_curve = {k:[] for k in recall_point}
    for qid, group in tqdm(dev_data.groupby('qid')):
        group = group.reset_index()
        rank = 0
        prev_score = None
        assert len(group['pid'].tolist()) == len(set(group['pid'].tolist()))
        # stable sort is also used in LightGBM
        total_rel = dev_rel_num.loc[qid]
        query_recall = [0 for k in recall_point]
        for t in group.sort_values('score', ascending=False, kind='mergesort').itertuples():
            if prev_score is not None and abs(t.score - prev_score) < 1e-8:
                score_tie_counter += 1
                score_tie_query.add(qid)
            prev_score = t.score
            rank += 1
            if t.rel>0:
                for i,p in enumerate(recall_point):
                    if rank <= p:
                        query_recall[i] += 1
        for i,p in enumerate(recall_point):
            if total_rel>0:
                recall_curve[p].append(query_recall[i]/total_rel)
            else:
                recall_curve[p].append(0.)

    score_tie = f'score_tie occurs {score_tie_counter} times in {len(score_tie_query)} queries'
    print(score_tie)
    
    for k,v in recall_curve.items():
        avg = np.mean(v)
        print(f'recall@{k}:{avg}')


In [11]:
train_X = train_extracted['data'].loc[:, feature_name]
train_Y = train_extracted['data']['rel']
dev_X = dev_extracted['data'].loc[:, feature_name]
dev_Y = dev_extracted['data']['rel']
lgb_train = lgb.Dataset(train_X,label=train_Y,group=train_extracted['group'])
lgb_valid = lgb.Dataset(dev_X,label=dev_Y,group=dev_extracted['group'])

In [12]:
for i,n in enumerate(feature_name):
    if np.isnan(train_X.iloc[:,i]).any():
        print(n)

contents_DPH
contents_TFIDF_confidence
contents_SCQ_confidence
contents_NormalizedTF_confidence
contents_IDF_confidence
contents_ICTF_confidence
expansion_TFIDF_confidence
expansion_SCQ_confidence
expansion_NormalizedTF_confidence
expansion_IDF_confidence
expansion_ICTF_confidence


In [13]:
dev_rel_num = dev_qrel[dev_qrel['rel']>0].groupby('qid').count()['rel']
gid = 0
sample_id = 0
dev_group_rel_num = []
for qid,group in dev_extracted['data'].groupby('qid'):
    group = group.sort_values(['pid'])
    assert len(group) == dev_extracted['group'].iloc[gid]
    assert np.isclose(group.iloc[0,:].loc[feature_name],
                      dev_X.iloc[sample_id,:], equal_nan=True).all()
    dev_group_rel_num.append(dev_rel_num.loc[qid])
    gid += 1
    sample_id += len(group)
dev_group=dev_extracted['group']

In [14]:
def recall_at_200(preds, dataset):
    global dev_group_rel_num
    global dev_group
    labels = dataset.get_label()
    groups = dataset.get_group()
    assert np.equal(groups, dev_group).all()
    idx = 0
    recall = 0
    for g,gnum in zip(groups, dev_group_rel_num):
        top_preds = labels[idx:idx+g][np.argsort(preds[idx:idx+g])]
        recall += np.sum(top_preds[-200:])/gnum
        idx += g
    assert idx == len(preds)
    return 'recall@200', recall/len(groups), True

In [15]:
dev_extracted['data']['score'] = 0.
for seed in [12345]:
    params = {
            'boosting_type': 'gbdt',
            'objective': 'lambdarank',
            'max_bin':255,
            'num_leaves':63,
            'max_depth':-1,
            'min_data_in_leaf':30,
            'min_sum_hessian_in_leaf':0,
#             'bagging_fraction':0.8,
#             'bagging_freq':50,
            'feature_fraction':1,
            'learning_rate':0.1,
            'num_boost_round':1000,
            'early_stopping_round':200,
            'metric':'custom',
            'label_gain':[0,1],
            'lambdarank_truncation_level':20,
            'seed':seed,
            'num_threads':max(multiprocessing.cpu_count()//2,1)
    }
    num_boost_round = params.pop('num_boost_round')
    early_stopping_round = params.pop('early_stopping_round')
    gbm = lgb.train(params, lgb_train,
                    valid_sets=lgb_valid,
                    num_boost_round=num_boost_round,
                    early_stopping_rounds=early_stopping_round,
                    feval=recall_at_200,
                    feature_name=feature_name,
                    verbose_eval=True)
    dev_extracted['data']['score'] += gbm.predict(dev_X)
    best_score = gbm.best_score['valid_0']['recall@200']
    print(best_score)
    best_iteration = gbm.best_iteration
    print(best_iteration)
    feature_importances = sorted(list(zip(feature_name,gbm.feature_importance().tolist())),
                                 key=lambda x:x[1],reverse=True)
    print(feature_importances)
eval_recall(dev_qrel, dev_extracted['data'])
eval_mrr(dev_extracted['data'])

[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.625544 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 33992
[LightGBM] [Info] Number of data points in the train set: 5298347, number of used features: 155
[1]	valid_0's recall@200: 0.858214
Training until validation scores don't improve for 200 rounds
[2]	valid_0's recall@200: 0.874546
[3]	valid_0's recall@200: 0.875681
[4]	valid_0's recall@200: 0.877853
[5]	valid_0's recall@200: 0.87999
[6]	valid_0's recall@200: 0.881853
[7]	valid_0's recall@200: 0.883202
[8]	valid_0's recall@200: 0.883477
[9]	valid_0's recall@200: 0.883572
[10]	valid_0's recall@200: 0.883644
[11]	valid_0's recall@200: 0.883668
[12]	valid_0's recall@200: 0.884157
[13]	valid_0's recall@200: 0.884444
[14]	valid_0's recall@200: 0.883942
[15]	valid_0's recall@200: 0.884873
[16]	valid_0's recall@200: 0.884659
[17]	valid_0

[215]	valid_0's recall@200: 0.893863
[216]	valid_0's recall@200: 0.893911
[217]	valid_0's recall@200: 0.89384
[218]	valid_0's recall@200: 0.89384
[219]	valid_0's recall@200: 0.894054
[220]	valid_0's recall@200: 0.893911
[221]	valid_0's recall@200: 0.894054
[222]	valid_0's recall@200: 0.893768
[223]	valid_0's recall@200: 0.893625
[224]	valid_0's recall@200: 0.89384
[225]	valid_0's recall@200: 0.893696
[226]	valid_0's recall@200: 0.893696
[227]	valid_0's recall@200: 0.893696
[228]	valid_0's recall@200: 0.893768
[229]	valid_0's recall@200: 0.893481
[230]	valid_0's recall@200: 0.893481
[231]	valid_0's recall@200: 0.893577
[232]	valid_0's recall@200: 0.893577
[233]	valid_0's recall@200: 0.893505
[234]	valid_0's recall@200: 0.893505
[235]	valid_0's recall@200: 0.893219
[236]	valid_0's recall@200: 0.893219
[237]	valid_0's recall@200: 0.893219
[238]	valid_0's recall@200: 0.893219
[239]	valid_0's recall@200: 0.89329
[240]	valid_0's recall@200: 0.893434
[241]	valid_0's recall@200: 0.893362
[242]

[438]	valid_0's recall@200: 0.89501
[439]	valid_0's recall@200: 0.894723
[440]	valid_0's recall@200: 0.894723
[441]	valid_0's recall@200: 0.894723
[442]	valid_0's recall@200: 0.89458
[443]	valid_0's recall@200: 0.894436
[444]	valid_0's recall@200: 0.894293
[445]	valid_0's recall@200: 0.894293
[446]	valid_0's recall@200: 0.89458
[447]	valid_0's recall@200: 0.89458
[448]	valid_0's recall@200: 0.89458
[449]	valid_0's recall@200: 0.894723
[450]	valid_0's recall@200: 0.894723
[451]	valid_0's recall@200: 0.894866
[452]	valid_0's recall@200: 0.894866
[453]	valid_0's recall@200: 0.894866
[454]	valid_0's recall@200: 0.89458
[455]	valid_0's recall@200: 0.89458
[456]	valid_0's recall@200: 0.89458
[457]	valid_0's recall@200: 0.89458
[458]	valid_0's recall@200: 0.89458
[459]	valid_0's recall@200: 0.89458
[460]	valid_0's recall@200: 0.89458
[461]	valid_0's recall@200: 0.89458
[462]	valid_0's recall@200: 0.894723
[463]	valid_0's recall@200: 0.894723
[464]	valid_0's recall@200: 0.894723
[465]	valid_0'

[661]	valid_0's recall@200: 0.894783
[662]	valid_0's recall@200: 0.894926
[663]	valid_0's recall@200: 0.894926
[664]	valid_0's recall@200: 0.894926
[665]	valid_0's recall@200: 0.894926
[666]	valid_0's recall@200: 0.894926
[667]	valid_0's recall@200: 0.894783
[668]	valid_0's recall@200: 0.894783
[669]	valid_0's recall@200: 0.894783
[670]	valid_0's recall@200: 0.894854
[671]	valid_0's recall@200: 0.894998
[672]	valid_0's recall@200: 0.894854
[673]	valid_0's recall@200: 0.894854
[674]	valid_0's recall@200: 0.894854
[675]	valid_0's recall@200: 0.894854
[676]	valid_0's recall@200: 0.894854
[677]	valid_0's recall@200: 0.894854
[678]	valid_0's recall@200: 0.894998
[679]	valid_0's recall@200: 0.894998
[680]	valid_0's recall@200: 0.894854
[681]	valid_0's recall@200: 0.894854
[682]	valid_0's recall@200: 0.894854
[683]	valid_0's recall@200: 0.894854
[684]	valid_0's recall@200: 0.894854
[685]	valid_0's recall@200: 0.894998
[686]	valid_0's recall@200: 0.894998
[687]	valid_0's recall@200: 0.894998
[

100%|██████████| 6980/6980 [03:13<00:00, 36.15it/s]


score_tie occurs 103315 times in 1822 queries
recall@10:0.5775429799426934
recall@20:0.6819126074498567
recall@50:0.7873447946513848
recall@100:0.8468481375358167
recall@200:0.8954393505253103
recall@500:0.9301337153772683
recall@1000:0.9470869149952243


100%|██████████| 6980/6980 [01:17<00:00, 90.55it/s] 


score_tie occurs 5 times in 5 queries
0.3013358461818347
